In [1138]:
import pandas as pd
import numpy as np
imdb_df = pd.read_csv("./MovieData/movie_metadata 2.csv")

In [1139]:
imdb_df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [1140]:
imdb_df.keys()

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

In [1141]:
imdb_df['title_year'].max()

2016.0

In [1142]:
df_movie_imdb_link = imdb_df['movie_imdb_link']

In [1143]:
movi_list = df_movie_imdb_link.to_list()

In [1144]:
df_movie_imdb_link

0       http://www.imdb.com/title/tt0499549/?ref_=fn_t...
1       http://www.imdb.com/title/tt0449088/?ref_=fn_t...
2       http://www.imdb.com/title/tt2379713/?ref_=fn_t...
3       http://www.imdb.com/title/tt1345836/?ref_=fn_t...
4       http://www.imdb.com/title/tt5289954/?ref_=fn_t...
                              ...                        
5038    http://www.imdb.com/title/tt3000844/?ref_=fn_t...
5039    http://www.imdb.com/title/tt2071645/?ref_=fn_t...
5040    http://www.imdb.com/title/tt2107644/?ref_=fn_t...
5041    http://www.imdb.com/title/tt2070597/?ref_=fn_t...
5042    http://www.imdb.com/title/tt0378407/?ref_=fn_t...
Name: movie_imdb_link, Length: 5043, dtype: object

In [1145]:
box_office = pd.read_csv("./MovieData/boxofficemojo_dataset/Mojo_budget_data.csv")

In [1146]:
box_office.keys()

Index(['movie_id', 'movie_title', 'movie_year', 'director', 'writer',
       'producer', 'composer', 'cinematographer', 'main_actor_1',
       'main_actor_2', 'main_actor_3', 'main_actor_4', 'budget', 'domestic',
       'international', 'worldwide', 'mpaa', 'run_time', 'genre_1', 'genre_2',
       'genre_3', 'genre_4', 'link'],
      dtype='object')

In [1147]:
imdb_df["movie_id"] = imdb_df.movie_imdb_link.str[26:35]


In [1148]:
imdb_df["movie_id"].head()        

0    tt0499549
1    tt0449088
2    tt2379713
3    tt1345836
4    tt5289954
Name: movie_id, dtype: object

In [1149]:
merged_df = pd.merge(imdb_df, box_office, on='movie_id')

In [1150]:
mask = merged_df.country.str.contains('USA')
merged_df = merged_df.drop(index=merged_df[~mask].index)

In [1151]:
merged_df = merged_df.drop(['main_actor_2', 'main_actor_3', 'main_actor_1', 'facenumber_in_poster'], axis=1)
# removing duplicates and urelated attributes
merged_df = merged_df.drop(['run_time', 'budget_y', 'director_name', 'title_year', 'link', 'movie_imdb_link', 'genres', 'num_voted_users', 'movie_title_y', 'num_critic_for_reviews', 'content_rating', 'country', 'aspect_ratio'], axis=1)

# merged_df = merged_df.drop(['genre_4', 'genre_3'], axis=1)

In [1152]:
merged_df['genre_2'].isna().sum()

137

In [1153]:
merged_df['movie_year'].min()

2000

In [1154]:
merged_df[['budget_x', 'movie_year']].head()

,budget_x,movie_year
0,237000000.0,2009
1,300000000.0,2007
3,250000000.0,2012
4,263700000.0,2012
5,258000000.0,2007


In [1155]:
# The dollar had an average inflation rate of 2.10% per year between 2000 and 2016. Source: https://www.officialdata.org/2000-dollars-in-2016?amount=3900#:~:text=The%20dollar%20had%20an%20average,Labor%20Statistics%20consumer%20price%20index.
inflation = 1.021
for index, row in merged_df.iterrows():
    power = row['movie_year'] - 2000
    merged_df.at[index, 'budget_x'] = row['budget_x'] * (inflation ** power)
    merged_df.at[index, 'worldwide'] = row['worldwide'] * (inflation ** power)
    merged_df.at[index, 'domestic']= row['domestic'] * (inflation ** power)
    merged_df.at[index, 'international']= row['international'] * (inflation ** power)
    merged_df.at[index, 'gross']= row['gross'] * (inflation ** power)

In [1156]:
merged_df[['budget_x', 'movie_year']].head()

,budget_x,movie_year
0,2.857459e+08,2009
1,3.469776e+08,2007
3,3.208108e+08,2012
4,3.383912e+08,2012
5,2.984007e+08,2007


In [1157]:
genre_1 = merged_df['genre_1'].unique()
genre_2 = merged_df['genre_2'].unique()
genre_3 = merged_df['genre_3'].unique()
genre_4 = merged_df['genre_4'].unique()


genres = np.concatenate((genre_1, genre_2, genre_3, genre_4), axis=0)

genres = genres[~pd.isna(genres)]
genres = np.unique(genres)

for g in genres:
    merged_df.insert(len(merged_df.columns), g, 0)

for item, row in merged_df.iterrows():
    for genre in genres:
        if genre == row['genre_1']:
            merged_df.at[item, genre] = 1
        if genre == row['genre_2']:
            merged_df.at[item, genre] = 1
        if genre == row['genre_3']:
            merged_df.at[item, genre] = 1
        if genre == row['genre_4']:
            merged_df.at[item, genre] = 1

In [1158]:
merged_df.head()

,color,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,actor_1_name,movie_title_x,cast_total_facebook_likes,...,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,Color,178.0,0.0,855.0,Joel David Moore,1000.0,9.169259e+08,CCH Pounder,Avatar,4834,...,0,0,0,0,0,1,0,0,0,0
1,Color,169.0,563.0,1000.0,Orlando Bloom,40000.0,3.578544e+08,Johnny Depp,Pirates of the Caribbean: At World's End,48350,...,0,0,0,0,0,0,0,0,0,0
3,Color,164.0,22000.0,23000.0,Christian Bale,27000.0,5.750605e+08,Tom Hardy,The Dark Knight Rises,106759,...,0,0,0,0,0,0,0,0,0,0
4,Color,132.0,475.0,530.0,Samantha Morton,640.0,9.375204e+07,Daryl Sabara,John Carter,1873,...,0,0,0,0,0,1,0,0,0,0
5,Color,156.0,0.0,4000.0,James Franco,24000.0,3.892283e+08,J.K. Simmons,Spider-Man 3,46055,...,0,0,0,0,0,1,0,0,0,0


In [1134]:
genres

array(['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror',
       'Music', 'Musical', 'Mystery', 'News', 'Romance', 'Sci-Fi',
       'Sport', 'Thriller', 'War', 'Western'], dtype=object)

In [1135]:
def normalize(df, feature):
    df[feature] = (df[feature]-df[feature].mean())/df[feature].std()
    # return df[feature].mean(), df[feature].std()

In [1136]:
len(merged_df)

1675

In [1159]:
# normalize(merged_df, 'domestic')
# normalize(merged_df, 'international')
# normalize(merged_df, 'worldwide')
# normalize(merged_df, 'gross')
normalize(merged_df, 'cast_total_facebook_likes')
normalize(merged_df, 'actor_1_facebook_likes')
normalize(merged_df, 'budget_x')
normalize(merged_df, 'actor_2_facebook_likes')
normalize(merged_df, 'movie_facebook_likes')
normalize(merged_df, 'director_facebook_likes')
normalize(merged_df, 'actor_3_facebook_likes')

In [1160]:
normalize(merged_df, 'duration')

In [1171]:
merged_df = merged_df.drop(['movie_year', 'movie_id', 'movie_title_x', 'num_user_for_reviews', 'plot_keywords'], axis=1)
merged_df = merged_df.drop(['domestic', 'international'], axis=1)
merged_df = merged_df.drop(['genre_2', 'genre_1', 'genre_3', 'genre_4'], axis=1)
merged_df = merged_df.drop(['gross'], axis=1)
merged_df = merged_df.drop(['imdb_score'], axis=1)

In [1163]:
mean_rev, std_rev = df['worldwide'].mean(), df['worldwide'].std()
normalize(merged_df, 'worldwide')

In [1112]:
# import seaborn as sns
# sns.heatmap(merged_df.corr(), cmap='YlGnBu', annot=True, linewidths = 0.2);

In [1164]:
from sklearn import preprocessing
def to_categorical(df, feature):
    le = preprocessing.LabelEncoder()
    le.fit(df[feature].unique())
    feature_cat = le.transform(df[feature])
    df.drop([feature], axis=1)
    df[feature] = feature_cat
    # return df

In [1165]:
to_categorical(merged_df, 'director')
to_categorical(merged_df, 'writer')
to_categorical(merged_df, 'producer')
to_categorical(merged_df, 'composer')
to_categorical(merged_df, 'cinematographer')
to_categorical(merged_df, 'main_actor_4')
to_categorical(merged_df, 'actor_2_name')
to_categorical(merged_df, 'actor_1_name')
to_categorical(merged_df, 'actor_3_name')
to_categorical(merged_df, 'mpaa')
to_categorical(merged_df, 'color')
to_categorical(merged_df, 'language')

In [1174]:
merged_df.dropna(inplace=True)

In [1175]:
# drop_list = ['worldwide', 'cast_total_facebook_likes', 'actor_1_facebook_likes', 'budget_x', 'actor_2_facebook_likes', 'movie_facebook_likes', 'director_facebook_likes', 'actor_3_facebook_likes', 'worldwide_normalized']
for i in drop_list:
    x = merged_df.loc[:, merged_df.columns != 'worldwide']
y = merged_df['worldwide']

In [1176]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 100)
mlr = LinearRegression()  
mlr.fit(x_train, y_train)

LinearRegression()

In [1181]:
#Prediction of test set
y_pred_mlr= mlr.predict(x_test)
#Predicted values
print("Prediction for test set: {}".format(y_pred_mlr))

Prediction for test set: [-6.96514927e-01 -6.11727579e-01 -2.61289544e-01 -9.12192917e-01
 -1.00485243e-01  1.78581161e-01  4.88628671e-01 -5.35927172e-01
 -6.27946929e-01  1.20016144e-01  3.80196403e-01 -4.35102026e-01
  7.02897727e-01 -4.72791340e-01 -5.52926918e-01  1.13555730e-01
 -2.00950921e-01 -8.87577828e-01  1.06416713e+00 -1.61814650e-01
  1.13934101e+00 -1.95538097e-01  1.48752574e+00 -4.34541278e-01
 -7.48291675e-01  3.19910588e-01  7.81226746e-02  7.98576267e-01
 -6.68581099e-02 -4.76772529e-02 -2.24040556e-01 -5.87181889e-01
 -5.94186007e-01  1.29119219e-01  4.22471435e-01  1.96506772e+00
 -2.96849559e-01 -5.05573498e-01  1.65053224e-02  7.83034035e-02
 -4.02265699e-01 -6.05491283e-01 -1.93868860e-01  9.20053317e-01
  2.89817062e-01 -2.84660024e-02 -6.30826500e-01  4.46826524e-01
  8.17296721e-01  1.04620371e-01  1.37514497e+00  1.82275576e-01
  3.97269583e-01  2.76327495e+00  1.97339317e-01 -3.43991343e-01
  6.29755145e-01 -7.48432787e-01  4.98101467e-01  6.96634245e-01


In [1182]:
from sklearn import metrics
meanAbErr = metrics.mean_absolute_error(y_test, y_pred_mlr)
meanSqErr = metrics.mean_squared_error(y_test, y_pred_mlr)
rootMeanSqErr = np.sqrt(metrics.mean_squared_error(y_test, y_pred_mlr))
print('R squared: {:.2f}'.format(mlr.score(x,y)*100))
print('Mean Absolute Error:', meanAbErr)
print('Mean Square Error:', meanSqErr)
print('Root Mean Square Error:', rootMeanSqErr)

R squared: 64.64
Mean Absolute Error: 0.37663412903469595
Mean Square Error: 0.35383159106835627
Root Mean Square Error: 0.5948374492820339


In [1183]:
mlr_diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': y_pred_mlr})
mlr_diff.head()

,Actual value,Predicted value
1835,-0.709269,-0.696515
1799,-0.370525,-0.611728
1031,-0.046411,-0.261290
1960,-0.577758,-0.912193
1234,-0.477775,-0.100485


In [937]:
merged_df['worldwide'].mean()

195333821.53679413

In [1180]:
x['worldwide']

KeyError: 'worldwide'

In [1123]:
x['budget_x']

0       2.857459e+08
1       3.469776e+08
3       3.208108e+08
4       3.383912e+08
5       2.984007e+08
            ...     
1981    1.132803e+05
1982    1.365797e+05
1983    7.063441e+04
1984    7.606783e+03
1985    1.195352e+03
Name: budget_x, Length: 1648, dtype: float64